# Mobile Robotics Project - Final Report

Group Members:
- ESCOYEZ, Antoine Jacques Richard, 335564
- POUSSIN, Jean-Baptiste Marie Alexandre, 303127
- KOCKISCH, Matthias Hugues Jörg, 303000
- STUBER, Lukas, 289304

Professor:

Prof. MONDADA, Francesco


EPFL, 11.12.2022
Course MICRO-452 Basics of Mobile Robotics

# Project Description
The goal of the project in the Basics of Mobile Robotics class was the control of a Thymio robot, a two wheeled robot. The environment of the robot is a map designed by the group members. On the map there need to be obstacles into which the robot cannot/must not drive. According to the chosen map and the obstacles present on the map a global path from the start to the goal needs to be calculated using computer vision (via a camera ?????). The start is the initial location of the Thymio, the goal can be chosen by the user. On its path to the goal the robot needs to be able to detect and avoid unforeseen obstacles, that are added by the user. For better tracking of the path some kind of filtering must be used that improves the state estimation of the robot. The camera image serves as measurement for the filter.

# Detailed Description of our Choices
Our group chose the map to be a parking space with two rows of parking lots. The yellow lines marking the edges of the parking lots are the fixed obstacles used in the global path planning. The robot is neighter allowed to cross the yellow lines nor leave the map. The camera is fixed above the map and oversees the entire situation (robot, lines, free spaces, occupied spaces). The goal is a free parking lot indicated by a red paper square. The unknown obstacles (unknown to/ignored by the computer vision) are ?????. The estimated state correction is done with the help of a Kalman filter whose measurement is the location of the Thymio in the camera images.

<font color='red'>Please replace the image with the new map</font>

<img src="Map.jpeg" width="300">